In [1]:
import numpy as np

In [2]:
# Enter the number of layers of your network
num_layers = int(input("Enter the number of layers : "))

# Enter the number of neurons of your network
split_int = lambda x : int(x.split(' '))
num_neurons = input("Enter the number of neurons of each layer in your network")

split_int(num_neurons)

5


In [14]:
# Construct a list that contains the number of neurons of each layer in your network.
structure = input("Enter the number of neurons of each layer in your network").split(' ')
structure = [int(x) for x in structure]
print(structure)    

[2, 5, 3]


In [19]:
# Construct a list that contains all the bias vectors of your network(_n means it has all vectors)
B_n = [np.random.randn(l, 1) for l in structure[1:]]

print("bias vector of each layer\n")
for B in B_n:
    print(B)
    print()

# Construct a list that contains all the weight matrices of your network(_n means it has all matrices)
W_n = [np.random.randn(l, next_l) for l, next_l in zip(structure[:-1], structure[1:])]

print("weight matrix of each layer\n")
for W in W_n:
    print(W)
    print()

bias vector of each layer

[[-2.32202785]
 [-0.04901366]
 [ 0.22587579]
 [-0.2931704 ]
 [ 1.01739342]]

[[ 2.01690202]
 [ 1.830421  ]
 [-2.05348695]]

weight matrix of each layer

[[ 0.59716928 -1.62108939 -0.22605826  0.3382741  -2.54043146]
 [ 0.88824668 -0.11187268 -0.72703204 -2.72262173  0.07305009]]

[[ 0.85730031 -0.56459314 -0.28259055]
 [ 0.9084166  -1.4537277   0.09349522]
 [-0.55943577  1.34928798 -0.24278885]
 [-0.63823328 -0.03144898  0.80522977]
 [-0.11595886 -0.389975   -0.06478941]]

